#EPH - Individuos


In [ ]:
import sys
import os
import csv
from pathlib import Path

Antes de comenzar, se establece la ruta a la fuente para no tener problemas en la importación de módulos.


In [ ]:
# Ajustar la ruta para que Python reconozca la carpeta "src"
project_path = os.path.abspath(os.path.join(".."))  
sys.path.append(project_path) 
# Verificación (debe mostrar la ruta a "src/")
print("Ruta de 'src':", os.path.join(project_path, "src"))

## Unificación del archivo

In [ ]:
from src.utils.helpers import  process_file, save_to_txt
# Ruta los archivos 
source_path = "../data/raw"
destination_path = "../data/clean"

# Procesar y guardar individuos
individual=process_file(source_path, category="individual")
save_to_txt(individual, destination_path, "individuos_unificado.txt")